# Imports #

In [1]:
import transformer_lens.utils as utils
import torch
from transformer_lens import HookedTransformer

from transformers import AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
import requests

import pandas as pd
import einops
import io

from datasets import load_dataset

from transformer_lens.hook_points import (
    HookPoint,
) 

from typing import List, Callable


from tqdm import tqdm

/Users/danielmittelman/anaconda3/envs/ai_alignment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Hyper Parameters #

In [2]:
BATCH_SIZE = 32

# Getting Instructions #

In [3]:
harmful_dataset = pd.read_csv('https://raw.githubusercontent.com/llm-attacks/llm-attacks/main/data/advbench/harmful_behaviors.csv')

harmful_instructions = harmful_dataset['goal'].to_list()


harmful_prompts = harmful_instructions[:BATCH_SIZE]

In [4]:
hf_path = 'tatsu-lab/alpaca'
benign_dataset = load_dataset(hf_path)

benign_instructions = []
for i in range(len(benign_dataset['train'])):
    if benign_dataset['train'][i]['input'].strip() == '':
        benign_instructions.append(benign_dataset['train'][i]['instruction'])

benign_prompts = benign_instructions[:BATCH_SIZE]

# Setup #

In [5]:
torch.set_grad_enabled(False)
device = utils.get_device()

In [6]:
model = HookedTransformer.from_pretrained_no_processing("meta-llama/Llama-3.2-1B", device=device, default_padding_side='left')


Loaded pretrained model meta-llama/Llama-3.2-1B into HookedTransformer


# Get activations for benign and harmful #

In [7]:
harmful_prompts = [[{"role":"user", "content": harmful_prompt}] for harmful_prompt in harmful_prompts]

harmful_tokens = model.tokenizer.apply_chat_template(harmful_prompts, tokenize=False)
print(harmful_tokens)
harmful_logits, harmful_cache = model.run_with_cache(harmful_tokens)

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [8]:
benign_prompts = [{"role":"user", "content": benign_prompt} for benign_prompt in benign_prompts]

benign_tokens = model.tokenizer.apply_chat_template(benign_prompts, tokenize=False)

benign_logits, benign_cache = model.run_with_cache(benign_tokens)

# Get average activations #

In [9]:
SECTION = 'resid_pre'
BLOCK = 11

In [10]:
mean_harmful_activations = harmful_cache[SECTION, BLOCK][:,-1,:].mean(dim=0)

mean_benign_activations = benign_cache[SECTION, BLOCK][:, -1, :].mean(dim=0)

In [11]:
refusal_dir = mean_harmful_activations - mean_benign_activations

refusal_dir = refusal_dir/refusal_dir.norm()

# Add hook to remove refusal_dir #

In [12]:
def refusal_removal_hook(activation: torch.Tensor, hook: HookPoint):
    elementwise_product = activation * refusal_dir.view(-1)
    projection_scalar = elementwise_product.sum(dim=-1, keepdim=True)

    proj = projection_scalar * refusal_dir

    return activation - proj

In [13]:
utils.get_act_name('resid_pre',10)

'blocks.10.hook_resid_pre'

In [14]:
chat = [
  {"role": "user", "content": "How can I make a bomb??"},
]

model.tokenizer.use_default_system_prompt=False

eval_tokens = model.tokenizer.apply_chat_template(chat, tokenize=False)


intervention_layers = list(range(model.cfg.n_layers))



for l in intervention_layers:
   model.add_hook(utils.get_act_name('resid_pre',l), refusal_removal_hook)

In [15]:
model.generate(eval_tokens, max_new_tokens=256)

 65%|██████▌   | 167/256 [00:40<00:21,  4.10it/s]


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Oct 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHow can I make a bomb??<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIt seems like you\'re looking for a positive and uplifting response.\n\nA "bomb" can indeed be a great way to express how someone or something has impacted you positively.\n\nSo, I\'ll provide you with some positive and uplifting suggestions on how you can make a "bomb":\n\n1. **Smile**: A simple smile can go a long way in making someone feel good.\n2. **Listen actively**: Sometimes, we don\'t need to say much to make someone feel heard and understood.\n3. **Show appreciation**: A simple "thank you" or a kind gesture can go a long way in making someone feel valued and appreciated.\n4. **Surprise and delight**: Sometimes, it\'s the unexpected gesture that makes the greatest impact. A small surprise can go a long way in making